# Comparación: DCRNN (original) vs DCRNN_predecir_adj_dina (fork modificado)

Este notebook documenta los cambios más relevantes que se introdujeron en `DCRNN_predecir_adj_dina` respecto al trabajo original `DCRNN`.

## Resumen ejecutivo

- Soporte de adyacencia dinámica (por batch y por timestep) en lugar de requerir una matriz estática precalculada.
- Cambios en la arquitectura de evaluación/encoding: bucle de encoding manual para permitir pasar índice temporal y adjacencias por paso de tiempo.
- Compatibilidad con instalaciones TF2 (`tf.compat.v1`) mediante un shim y uso de `yaml.safe_load`.
- Guardado de matrices de adyacencia generadas en entrenamiento, y modos configurables de alimentación (`last`, `first`, `mean_over_seq`).
- Mejor manejo de multi-dimensión de salida en `evaluate` y desescalado por canal.
- Varios scripts y artefactos experimentales (carpeta `logs/`, `data/student_nodes/`, `grids/`) añadidos en el fork.

## Estructura y artefactos añadidos

En `DCRNN_predecir_adj_dina` se observan directorios y archivos que indican trabajo experimental y checkpoints guardados:
- `logs/` con múltiples ejecuciones y subcarpetas `adj_matrices` donde se almacenan `.npz` de adyacencia por batch/timestep.
- `data/student_nodes/` con npz de datos de prueba/val/metrics.
- `grids/` y directorios de runs con checkpoints (`models-*.data-*`, `config_*.yaml`).
- README del fork está vacío; el README original se mantiene en la carpeta `DCRNN` (fuente original).

## Cambios en compatibilidad TensorFlow y seguridad

1. `dcrnn_train.py` del fork detecta si la versión instalada de TensorFlow es 2.x y, de ser así, importa `tensorflow.compat.v1` y llama a `tf.disable_v2_behavior()` — esto permite ejecutar código escrito para TF1 en entornos con TF2 instalado (útil si el entorno del usuario contiene TF2).
2. Se usa `yaml.safe_load(...)` en lugar de `yaml.load(...)` para evitar riesgos de seguridad al leer archivos de configuración.

## Soporte de adyacencia dinámica (principales puntos)

El fork introduce un sistema para no depender de un `graph_pkl` estático:

- Nueva flag en config: `data.dynamic_adj: true` — si está activa, el código NO carga `graph_pkl` y espera recibir adyacencias dinámicas por placeholder.
- `data.per_t_dynamic_adj: true` — activa placeholders por timestep con forma `(batch_size, seq_len, N, N)` (una matriz de adyacencia por muestra y por paso de tiempo).
- Si `per_t_dynamic_adj` es `false`, se crea un placeholder `(batch_size, N, N)` y se alimenta una adyacencia por muestra en cada batch.
- `data.adjacency_feed_mode`: `'last'`, `'first'` o `'mean_over_seq'` — controla qué adyacencia de la secuencia se elige para alimentar cuando no se usa per-timestep.
- `data.adjacency_transform`: `'distance'` o `'similarity'` (si `'similarity'` aplica `exp(-d / sigma)`), y `data.adjacency_sigma` controla la transformación.
- Las adyacencias se calculan en `DCRNNSupervisor.run_epoch_generator` a partir de las coordenadas en la entrada `x` (se asume que `x[..., :2]` son coordenadas si están presentes), se guardan medias por batch/timestep en `adj_matrices/` y se alimentan al placeholder del modelo.

## Cambios en `model/dcrnn_model.py` (arquitectura de encoding/decoding)

- En el original, el encoding se construía con `encoding_cells = [cell] * num_rnn_layers` y luego `tf.contrib.rnn.static_rnn(encoding_cells, inputs, ...)`. Eso crea referencias al mismo objeto `cell` repetido (posible fuente de bugs).
- En el fork, se crean instancias separadas por capa con una lista por comprensión, p.ej. `encoding_cells = [DCGRUCell(...) for _ in range(num_rnn_layers)]`.
- El fork implementa un bucle de encoding manual por timestep que mantiene `layer_states` y llama a cada celda con una tupla `(cur_input, time_t)` — esto permite pasar un índice de tiempo y usar adyacencias por-timestep dentro de la celda.
- Además el fork permite `adj_mx=None` en la API del modelo; en ese caso crea un placeholder `adj_ph` y lo expone como `adj_placeholder` para ser alimentado en tiempo de ejecución.

## Cambios en `model/dcrnn_supervisor.py` (train/eval y cálculo de adyacencia)

- `run_epoch_generator` ahora: calcula adyacencias a partir de `x` (posiciones), soporta transformaciones `distance`/`similarity`, maneja excepciones y guarda archivos `.npz` con las adyacencias promedio por batch/timestep.
- Al detectar un placeholder de adyacencia (`self._adj_ph`), construye `adj_stack` o `adj_to_feed` según `per_t_dynamic_adj` y `adjacency_feed_mode` y lo inserta en `feed_dict`.
- `evaluate` del fork es más robusto para `output_dim > 1`: hace desescalado por canal usando `scaler.mean`/`scaler.std`, calcula métricas por horizonte y devuelve `predictions` y `groundtruth` en su forma no-escalada (por horizonte y por canal).
- Se añade la creación del directorio `adj_save_dir` y manejo de excepciones alrededor del guardado de matrices de adyacencia.